# AutoML with AutoKeras

AutoKeras is an opensource AutoML framework based on Tensorflow/Keras. See the [documentation](https://autokeras.com/) for detailed information.

## Install AutoKeras

In [ ]:
!pip install git+https://github.com/keras-team/keras-tuner.git@1.0.2rc3
!pip install autokeras

## Load the MNIST Dataset

The MNIST data set is a database of handwritten digits that is commonly used for training various image processing systems. The goal if this task is to implement a classifier of handwritten digits using AutoKeras.

![Mnist data set](https://github.com/jirimaterna/image-processing-2days/blob/master/images/mnist-examples.png?raw=1)

In [ ]:
from tensorflow.keras.datasets import mnist
import autokeras as ak

(x_train, y_train), (x_test, y_test) = mnist.load_data()
print(x_train.shape) 
print(y_train.shape)

## Search for the best architecture.

In [ ]:
clf = ak.ImageClassifier( overwrite=True, max_trials=1)
#clf = ak.ImageClassifier( overwrite=True, max_trials=20)
clf.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=5)

## Search in vanilla models only

In [ ]:
input_node = ak.ImageInput()
output_node = ak.ImageBlock(block_type="vanilla", normalize=True, augment=False, )(input_node)
output_node = ak.ClassificationHead()(output_node)

clf = ak.AutoModel(inputs=input_node, outputs=output_node, overwrite=True, max_trials=20)
clf.fit(x_train, y_train, epochs=5)

## Export the neural network model

In [ ]:
model = clf.export_model()
print (model.summary())

## Export train the model for more epochs and evaluate

In [ ]:
import numpy as np
from tensorflow.python.keras.utils import np_utils

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

n_classes = 10
y_train_oh = np_utils.to_categorical(y_train, n_classes)
y_test_oh = np_utils.to_categorical(y_test, n_classes)

model.fit(x_train, y_train_oh, validation_data=(x_test, y_test_oh), epochs=20)

In [ ]:
from sklearn import metrics
from sklearn.metrics import accuracy_score

y_pred = model.predict(x_test)
y_pred_class = np.argmax(y_pred, axis=1)
print ("Test accuracy: {:.4f}".format(accuracy_score(y_test, y_pred_class)))
print ()
print(metrics.classification_report(y_test, y_pred_class, digits=4))